In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as agg
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [3]:
fs.ls("3di-data-argus")

['3di-data-argus/Argus_Merchant_Group_Hierarchy_2018-05-08.csv',
 '3di-data-argus/README_Merchant_Group_Hierarchy.txt',
 '3di-data-argus/UKCA_Behavioural_Modelling_Account_Level.txt',
 '3di-data-argus/UKCA_Behavioural_Modelling_Trans_Level.txt',
 '3di-data-argus/postcodes.csv']

In [15]:
fp = "s3://3di-data-argus/postcodes.csv"
df = ha.read_csv(fp, nrows=3)
df

,CustomerNumber,PostcodeDistrict
0,59671,BH06
1,1530724,BN02
2,2414080,BS25


In [14]:
fp = "s3://3di-data-argus/Argus_Merchant_Group_Hierarchy_2018-05-08.csv"
df = ha.read_csv(fp, nrows=3, header=1, usecols=range(1, 5))
df

,Argus Merchant Group,Argus Merchant Group Description,Category,Category Description
0,1,Airlines,3000,UNITED AIRLINES
1,1,Airlines,3001,AMERICAN AIRLINES
2,1,Airlines,3005,BRITISH AIRWAYS


In [27]:
fp = "s3://3di-data-argus/UKCA_Behavioural_Modelling_Account_Level.txt"
df = ha.read_csv(fp, nrows=3)
df
# df.groupby(['CustomerNumber', 'AccountNumber']).PeriodId.nunique()

,CustomerNumber,AccountNumber,PeriodId,AnnualFeeFlag,Cyc_Xxx_AprMerchant,Cyc_Xxx_AprCash,Cyc_Xxx_CreditLimitTotal,Cyc_Xxx_CycleEndDate,Cyc_Xxx_BalanceBeginning,Cyc_Xxx_BalanceEnding,Cyc_Xxx_BalanceAdb,Cyc_Xxx_BalanceRevolving,Cyc_Xxx_CyclesDelinquent,Cyc_Xxx_PaymentDueDate,Cyc_Xxx_PaymentMinimumDueAmount,Status,Cyc_Xxx_UnitChargeOffRate,OpenDate
0,850241,1482948,201304,0,0.159,0.279,5700,2013-04-12 00:00:00,139.67,6.90,155.9342,0,0,NaN,0,Open,0.012,2008-10-16 00:00:00
1,850241,1482948,201305,0,0.159,0.279,5700,2013-05-12 00:00:00,6.90,138.41,3.2200,0,0,NaN,0,Open,0.012,2008-10-16 00:00:00
2,850241,1482948,201306,0,0.159,0.279,5700,2013-06-14 00:00:00,138.41,138.41,138.4100,0,0,NaN,0,Open,0.016,2008-10-16 00:00:00


In [29]:
fp = "s3://3di-data-argus/UKCA_Behavioural_Modelling_Trans_Level.txt"
df = ha.read_csv(fp, nrows=100)
df

,CustomerNumber,AccountNumber,CyclePeriod,TransactionType,TransactionAmount,TransactionCount
0,1,1,201212,Finance Charges: Purchases,14.32,1
1,1,1,201301,Finance Charges: Purchases,14.49,1
2,1,1,201301,Payments: Non-DD,-40.00,1
3,1,1,201301,Purchases: MCC Group 9,19.99,1
4,1,1,201302,Finance Charges: Purchases,14.08,1
5,1,1,201302,Payments: Non-DD,-50.00,1
6,1,1,201303,Finance Charges: Purchases,12.41,1
7,1,1,201303,Payments: Non-DD,-30.00,1
8,1,1,201304,Finance Charges: Purchases,13.58,1
9,1,1,201304,Payments: Non-DD,-20.00,1
